In [1]:
## Hyper-parameter Tunning

In [9]:
#!pip install -q -U keras-tuner
import keras_tuner as kt

In [3]:
import tensorflow as tf
from tensorflow import keras


#helper libraries
#import numpy as np
#import matplotlib.pyplot as plt

#print (tf.__version__)

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [5]:
#normalize the format of pixel values to float and normalize the pixel values

img_train = img_train.astype('float32') / 255

img_test = img_test.astype('float32') / 255



Define the model:
\
when you build your model for hyper tuning,

for each hyperparameter to be tuned, you need to define a search range or search space
The tuner will use the values defined in search range to find the best value of hyperparameter

the model that you set up for hypertuning is called a hypermodel

Two approaches to define a hypermodel:

1.  using a model build function

2.  using predefined subclasses of hypermodel class of keras tuner API

two predefined subclasses:  HyperXception and HyperResNet

the model builder function returns us a compiled model and users hyperparameters you define inline to hypertune the model.

In [7]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    #model.add(keras.layers.Dense(128, activation='relu'))   Original line

    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)  # parameter I want to tune, + range of possible values
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))  # first parameter is what I want to tune.  ex.  units

    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice('learning_rate', values=[0.01,0.001,0.0001])  # parameter I want to tune, valid values 
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),  # only parameter i want to tune
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # convert output into probabilities array to do cross entropy
                metrics=['accuracy']
            ) #
    
    return model

# instantiate the tuner
Keras tuner library has 4 tuners to offer

1. RandomSearch
2. Hyperband
3. BayesianOptimization

In [10]:
# tuner builds and runs new models from all combinations between hyperparameters

tuner = kt.Hyperband(model_builder,
                objective='val_accuracy',  
                max_epochs=10,
                factor=3,
                directory='my_dir',
                project_name='intro_to_kt')

**early stopping**: helps you in controlling overfitting
creating callbacks to stop training early after reaching a certain value of validation loss

In [11]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 01m 09s]
val_accuracy: 0.8806666731834412

Best val_accuracy So Far: 0.8886666893959045
Total elapsed time: 00h 19m 24s
INFO:tensorflow:Oracle triggered exit


In [12]:
# get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

#hyperparameters
#number of nodes
# learning rate

In [14]:
print(f"""
the optimal number of units is {best_hps.get('units')}.
the best learning rate is {best_hps.get('learning_rate')}.
"""
)


the optimal number of units is 448.
the best learning rate is 0.001.



In [15]:
model = tuner.hypermodel.build(best_hps)
new_model = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

Epoch 1/50
1500/1500 [==============================] - 9s 5ms/step - loss: 0.4956 - accuracy: 0.8250 - val_loss: 0.3931 - val_accuracy: 0.8576
Epoch 2/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3685 - accuracy: 0.8658 - val_loss: 0.3649 - val_accuracy: 0.8678
Epoch 3/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3306 - accuracy: 0.8785 - val_loss: 0.3383 - val_accuracy: 0.8756
Epoch 4/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3049 - accuracy: 0.8872 - val_loss: 0.3414 - val_accuracy: 0.8767
Epoch 5/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2852 - accuracy: 0.8950 - val_loss: 0.3226 - val_accuracy: 0.8827
Epoch 6/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2700 - accuracy: 0.8992 - val_loss: 0.3449 - val_accuracy: 0.8802
Epoch 7/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2561 - accuracy: 0.9046 - val_loss: 0.3105 - val_accuracy:

In [16]:
# tune epochs value
val_acc_per_epoch = new_model.history['val_accuracy']

best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

print ('Best epoch: %d' %(best_epoch))

Best epoch: 47


In [17]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(img_train,label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/47
1500/1500 [==============================] - 12s 8ms/step - loss: 0.4933 - accuracy: 0.8257 - val_loss: 0.3891 - val_accuracy: 0.8602
Epoch 2/47
1500/1500 [==============================] - 15s 10ms/step - loss: 0.3689 - accuracy: 0.8637 - val_loss: 0.3554 - val_accuracy: 0.8725
Epoch 3/47
1500/1500 [==============================] - 11s 7ms/step - loss: 0.3291 - accuracy: 0.8779 - val_loss: 0.3447 - val_accuracy: 0.8773
Epoch 4/47
1500/1500 [==============================] - 10s 6ms/step - loss: 0.3034 - accuracy: 0.8875 - val_loss: 0.3164 - val_accuracy: 0.8843
Epoch 5/47
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2849 - accuracy: 0.8941 - val_loss: 0.3226 - val_accuracy: 0.8852
Epoch 6/47
1500/1500 [==============================] - 10s 7ms/step - loss: 0.2695 - accuracy: 0.9000 - val_loss: 0.3355 - val_accuracy: 0.8802
Epoch 7/47
1500/1500 [==============================] - 11s 7ms/step - loss: 0.2540 - accuracy: 0.9036 - val_loss: 0.3538 - val_ac